In [1]:
#EEG 经验模态分解
import sampleEntropy as se
from pyhht.visualization import plot_imfs
from pyhht.emd import EMD
import numpy as np
import scipy as sp
import pandas as pd
import pickle 
import matplotlib.pyplot as plt
import multiprocessing
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
#读取32个通道的EEG数据，每个通道包含32×40=1280个信号样本（人次×每人次40实验）
#每个样本向量大小为8064点（63s*128Hz）
for eeg_CH in range(1,33,1):
    file_path = "./dump_file/CH{}_df_EEG_x".format(eeg_CH)
    locals()['CH{}_df_EEG_x'.format(eeg_CH)] = pickle.load(open(file_path,"rb"))

In [3]:
'''pyhht自带的画图
t = range(0,8064,1)
x = CH1_df_EEG_x.iloc[20,:]
decomposer = EMD(x)
imfs = decomposer.decompose()
#plot_imfs(x, imfs, t)
'''
'''matplotlib画图
plt.figure(dpi=300)
plt.subplot(4,1,1)
plt.ylabel("EEG signal")
plt.plot(x)
for i in range(0,3,1):
    plt.subplot(4,1,i+2)
    plt.ylabel("IMF{}".format(i+1))
    plt.plot(imfs[i])
plt.show()
plt.figure(dpi=300)
for i in [3,4,5,6]:
    plt.subplot(4,1,i-2)
    plt.ylabel("IMF{}".format(i+1))
    plt.plot(imfs[i])
plt.show()
plt.figure(dpi=300)
for i in [7,8,9,10]:
    plt.subplot(4,1,i-6)
    if i==10:
        plt.ylabel("RES")        
    else:
        plt.ylabel("IMF{}".format(i+1))
    plt.plot(imfs[i])
plt.show()
'''

'matplotlib画图\nplt.figure(dpi=300)\nplt.subplot(4,1,1)\nplt.ylabel("EEG signal")\nplt.plot(x)\nfor i in range(0,3,1):\n    plt.subplot(4,1,i+2)\n    plt.ylabel("IMF{}".format(i+1))\n    plt.plot(imfs[i])\nplt.show()\nplt.figure(dpi=300)\nfor i in [3,4,5,6]:\n    plt.subplot(4,1,i-2)\n    plt.ylabel("IMF{}".format(i+1))\n    plt.plot(imfs[i])\nplt.show()\nplt.figure(dpi=300)\nfor i in [7,8,9,10]:\n    plt.subplot(4,1,i-6)\n    if i==10:\n        plt.ylabel("RES")        \n    else:\n        plt.ylabel("IMF{}".format(i+1))\n    plt.plot(imfs[i])\nplt.show()\n'

In [4]:
def runEMD(eegCH):
    #用来计算程序运行时间
    import datetime
    import pickle
    starttime = datetime.datetime.now()
    print("[CH{}]job start!".format(eegCH))
    file_path = "./dump_file/CH{}_df_EEG_x".format(eegCH)
    locals()['CH{}_df_EEG_x'.format(eegCH)] = pickle.load(open(file_path,"rb"))
    if eegCH != 2:
        for i in range(0,1280,1):
            x = locals()['CH{}_df_EEG_x'.format(eegCH)].iloc[i,:]
            decomposer = EMD(x)
            imfs = decomposer.decompose()
            file_path = "./dump_file_imfs/CH{}_imfs_{}".format(eegCH,i)
            pickle.dump(imfs,open(file_path,"wb"))
    else:
        dfCH2 = abs(locals()['CH{}_df_EEG_x'.format(eegCH)])
        for i in range(0,1280,1):
            x = dfCH2.iloc[i,:]
            decomposer = EMD(x)
            imfs = decomposer.decompose()
            file_path = "./dump_file_imfs/CH{}_imfs_{}".format(eegCH,i)
            pickle.dump(imfs,open(file_path,"wb"))        
    print("[CH{}]job done!".format(eegCH))
    #用来计算程序运行时间
    endtime = datetime.datetime.now()
    print("程序运行时间:%.1fs"%(endtime - starttime).seconds)
'''
if False:
    for eegCH in range(1,33,1):
        locals()["p{}".format(eegCH)] = multiprocessing.Process(target = runEMD, args = (eegCH,))
        locals()["p{}".format(eegCH)].start()
        print("p{}.pid:{}".format(eegCH,locals()["p{}".format(eegCH)].pid))
else:
    runEMD(eegCH=2)
'''       

'\nif False:\n    for eegCH in range(1,33,1):\n        locals()["p{}".format(eegCH)] = multiprocessing.Process(target = runEMD, args = (eegCH,))\n        locals()["p{}".format(eegCH)].start()\n        print("p{}.pid:{}".format(eegCH,locals()["p{}".format(eegCH)].pid))\nelse:\n    runEMD(eegCH=2)\n'

In [8]:
#使用原始EEG信号提取样本熵特征
def countSampleEntropy(eegCH,data,lineIndex):
    tmpSE = se.sampEn(data,m=2,r=1)
    filePath = "./dump_file_sampEn/CH{}_sampEn_{}".format(eegCH,lineIndex)
    pickle.dump(tmpSE,open(filePath,"wb"))
    print("[CH{}]->{} done!".format(eegCH,lineIndex))
    
def countSampleEntropySomes(eegCH,data,startIndex,endIndex):    
    sampEnList = []
    for i in range(0,endIndex-startIndex,1):
        std = np.std(data.iloc[i,:].values)
        try:
            tmpSampEn = se.sampEn(data.iloc[i,:].values,m=2,r=0.2*std)  #这里不能是dataframe
        except BaseException:
            print("ERROR!!!!!!!!!!!!!!!!!!",eegCH,startIndex,endIndex,i)          
        sampEnList.append(tmpSampEn)
    result = pd.DataFrame(sampEnList,index=data.index,columns=['CH{}SampEn'.format(eegCH)])
    filePath = "./dump_file_sampEn/CH{}_sampEn_{}_{}".format(eegCH,startIndex,endIndex)
    pickle.dump(result,open(filePath,"wb"))
    print("[CH{}]->{}_{} done!".format(eegCH,startIndex,endIndex))

cpuNums = 20
steps = 1280 // cpuNums
startIndexList = [idx for idx in range(0,1280,steps)]
startPoint = 6784
endPoint = 8064
if False:
    for eegCH in range(1,33,1):
        for sIdx in startIndexList:
            if eegCH !=2:
                if sIdx+steps >= 1280:
                    data = locals()['CH{}_df_EEG_x'.format(eegCH)].iloc[sIdx:1280,startPoint:endPoint]
                else:
                    data = locals()['CH{}_df_EEG_x'.format(eegCH)].iloc[sIdx:sIdx+steps,startPoint:endPoint]
                locals()["p{}".format(eegCH)] = multiprocessing.Process(target = countSampleEntropySomes, 
                                                                        args = (eegCH,data,sIdx,sIdx+steps))
                locals()["p{}".format(eegCH)].start()
                print("CH{}_sIdx{}.pid:{}".format(eegCH,sIdx,locals()["p{}".format(eegCH)].pid))
            else:#CH2是复数，另外处理
                data = abs(locals()['CH{}_df_EEG_x'.format(eegCH)])
                if sIdx+steps >= 1280:
                    data = data.iloc[sIdx:1280,startPoint:endPoint]
                else:
                    data = data.iloc[sIdx:sIdx+steps,startPoint:endPoint]
                locals()["p{}".format(eegCH)] = multiprocessing.Process(target = countSampleEntropySomes, 
                                                                        args = (eegCH,data,sIdx,sIdx+steps))
                locals()["p{}".format(eegCH)].start()
                print("CH{}_sIdx{}.pid:{}".format(eegCH,sIdx,locals()["p{}".format(eegCH)].pid))


In [9]:
#读取样本熵文件,将每个通道分别写入文件中
if True:   
    for eegCH in range(1,33,1):
        sampEnDF = pd.DataFrame()
        for sIdx in startIndexList:
            if sIdx+steps >= 1280:
                filePath = "./dump_file_sampEn/CH{}_sampEn_{}_{}".format(eegCH,sIdx,1280)
                locals()["CH{}_sampEn_{}_{}".format(eegCH,sIdx,1280)] = pickle.load(open(filePath,"rb"))
                sampEnDF = pd.concat([sampEnDF,locals()["CH{}_sampEn_{}_{}".format(eegCH,sIdx,1280)]],axis=0)
            else:
                filePath = "./dump_file_sampEn/CH{}_sampEn_{}_{}".format(eegCH,sIdx,sIdx+steps)
                locals()["CH{}_sampEn_{}_{}".format(eegCH,sIdx,sIdx+steps)] = pickle.load(open(filePath,"rb"))
                sampEnDF = pd.concat([sampEnDF,locals()["CH{}_sampEn_{}_{}".format(eegCH,sIdx,sIdx+steps)]],axis=0)
        filePath = "./dump_file_sampEn/CH{}_sampEn".format(eegCH)
        pickle.dump(sampEnDF,open(filePath,"wb"))
        

In [10]:
if True:
    sampEnDF = pd.DataFrame()
    for eegCH in range(1,33,1):
        filePath = "./dump_file_sampEn/CH{}_sampEn".format(eegCH)
        tmpDF = pickle.load(open(filePath,"rb"))
        sampEnDF = pd.concat([sampEnDF,tmpDF],axis=1)
    filePath = "./dump_file_sampEn/sampEnFeatures".format(eegCH)
    pickle.dump(sampEnDF,open(filePath,"wb"))
    


In [12]:
sampEnDF.shape


(1280, 32)